In [ ]:
# update bad images status 

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy.stats import gamma
from scipy.optimize import curve_fit
from scipy import special

from configparser import ConfigParser, ExtendedInterpolation

import utils as utils

from tqdm import tqdm_notebook as tqdm

In [ ]:
pd.set_option('display.max_columns', 50)

In [ ]:
# read config file 
config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('config.ini')

In [ ]:
FOLDER = config.get("all", "FOLDER")
DB_FILENAME = config.get("all", "DB_FILENAME")
DB_NEW_FILENAME = config.get("all", "DB_NEW_FILENAME")
# EXPERIMENTS = ["N2", "SEA-12", "MK4", "CB428", "RNAi"]
CHANNELS = ["C0-", "C1-", "C2-"]
SMFISH_COLUMNS = ["#c0_smfish", "#c1_smfish", "#c2_smfish"]

In [ ]:
EXPERIMENTS = ['N2', 'SEA-12', 'MK4', 'CB428', 'RNAi']
CHANNELS = ['C0-', 'C1-', 'C2-', 'C3-', 'C4-']
SMFISH_COLUMNS = ['#c0_smfish', '#c1_smfish', '#c2_smfish', '#c3_smfish', '#c4_smfish']

In [ ]:
# read the db and parse images that we want to process
df_path = os.path.join(FOLDER, "smFISH-database", DB_FILENAME)
df = pd.read_csv(df_path, 
                 sep=',',
                 na_values=[''],
                 # index_col=['cropped_image_file'],
            )


In [ ]:
df = df[[
    '#c0_smfish',
    '#c0_smfish_adj',
    '#c1_smfish',
    '#c1_smfish_adj',
    '#c2_smfish',
    '#c2_smfish_adj',
    '#channels',
    '#nuclei',
    '#nucs_predicted',
    'DAPI channel',
    'GFP channel',
    'c0',
    'c0_lambda',
    'c0_type',
    'c1',
    'c1_lambda',
    'c1_type',
    'c2',
    'c2_lambda',
    'c2_type',
    'c3',
    'c3_lambda',
    'c4',
    'c4_lambda',
    'crop_offset_x',
    'crop_offset_y',
    'cropped_image_file',
    'cropped_mask_file',
    'ellipse',
    'filename',
    'is_dapi_stack',
    'is_male',
    'is_male_batch',
    'is_too_bleached',
    'is_valid_final',
    'is_z_cropped',
    'num_z_planes',
    'original filename',
    'signal',
    'status',
    'tx',
    'tx_desc',
    'unique_id',
]]

In [ ]:
df.head()

In [ ]:
# update happens here

In [ ]:
dataset = []
for r, d, f in os.walk(os.path.join(FOLDER, 'bad')):
    for file in f:
        if '.csv' in file:
            dataset.append(file)

In [ ]:
dff = df.copy()

In [ ]:
pbar = tqdm(total=len(dataset))
for f in dataset:
    pbar.update(1)
    
    smfish_column = '#' + f[:3].lower()[:2] + '_smfish'    
    index = df[df['cropped_image_file'] == f[3:-4] + '.tif'].index
    
    
    print(smfish_column, index)
    
    df.at[index, smfish_column] = -3
    
pbar.close()

In [ ]:
df

In [ ]:
for col in [
        # '#channels',
        '#nuclei',
        '#nucs_predicted',
        'DAPI channel',
        'GFP channel',
        'c4_lambda',
        'crop_offset_x',
        'crop_offset_y',
        'is_dapi_stack',
        'is_male',
        'is_male_batch',
        'is_too_bleached',
        'is_valid_final',
        'is_z_cropped',
        'num_z_planes',
        'signal',
        'status',
        'tx',
        'tx_desc',
        # 'unique_id',
    ]:
    dff[col] = dff[col].astype(np.float32)

In [ ]:
for col in [
        'unique_id',
    ]:
    dff[col] = dff[col].astype(np.int32)

In [ ]:
dff.to_csv(
    os.path.join(FOLDER, "smFISH-database", DB_NEW_FILENAME), 
    index=False,
    columns=[
        '#c0_smfish',
        '#c0_smfish_adj',
        '#c1_smfish',
        '#c1_smfish_adj',
        '#c2_smfish',
        '#c2_smfish_adj',
        '#channels',
        '#nuclei',
        '#nucs_predicted',
        'DAPI channel',
        'GFP channel',
        'c0',
        'c0_lambda',
        'c0_type',
        'c1',
        'c1_lambda',
        'c1_type',
        'c2',
        'c2_lambda',
        'c2_type',
        'c3',
        'c3_lambda',
        'c4',
        'c4_lambda',
        'crop_offset_x',
        'crop_offset_y',
        'cropped_image_file',
        'cropped_mask_file',
        'ellipse',
        'filename',
        'is_dapi_stack',
        'is_male',
        'is_male_batch',
        'is_too_bleached',
        'is_valid_final',
        'is_z_cropped',
        'num_z_planes',
        'original filename',
        'signal',
        'status',
        'tx',
        'tx_desc',
        'unique_id',
    ]
)